In [1]:
# Imports

# Usual imports
import os
import sys
import time
import datetime
import json
import csv
import glob
import pprint
import importlib
from tqdm import tqdm
from collections import Counter

import quandl
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

import itertools
import time
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

%matplotlib inline
plt.style.use('seaborn-talk')

# Plotting
import matplotlib.pyplot as plt
from matplotlib import cm, style
from matplotlib.ticker import FixedLocator
from matplotlib.dates import DateFormatter

import seaborn as sns
from IPython.display import Image
from lifelines import KaplanMeierFitter
from lifelines import NelsonAalenFitter

# For regression
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

# Custom modules
sys.path.insert(0, '../')
#import psychometry
import parser
import utils

# Some edits to matplotlib
# https://towardsdatascience.com/simple-ways-to-improve-your-matplotlib-b64eebccfd5
# https://medium.com/@andykashyap/top-5-tricks-to-make-plots-look-better-9f6e687c1e08
#plt.rcParams.update({'font.size': 20})
style.use('seaborn-talk') #sets the size of the charts

# This is the color of sh.rice.edu/covid-sense website background. It is NOT white
bg_color = (0.96862745, 0.96862745, 0.96862745)

# Convert a timestamp into time string
d2s = lambda x : datetime.datetime.fromtimestamp(x/1000.0).strftime('%B %d')
sns.color_palette("Set2")


[(0.4, 0.7607843137254902, 0.6470588235294118),
 (0.9882352941176471, 0.5529411764705883, 0.3843137254901961),
 (0.5529411764705883, 0.6274509803921569, 0.796078431372549),
 (0.9058823529411765, 0.5411764705882353, 0.7647058823529411),
 (0.6509803921568628, 0.8470588235294118, 0.32941176470588235),
 (1.0, 0.8509803921568627, 0.1843137254901961),
 (0.8980392156862745, 0.7686274509803922, 0.5803921568627451),
 (0.7019607843137254, 0.7019607843137254, 0.7019607843137254)]

In [2]:
# Load database
dbname = '../data/db_snapshot.json'
import os
cwd = os.getcwd()
print(cwd)
with open(dbname, 'rb') as readfile:
    db = json.load(readfile)

lens = np.array([len(db[key]['timestamps']) for key in db])
print(lens.sum())

/Users/mahsanabbasi/Desktop/CE-EmaResponseAnalyzer-master-4/analysis
18783


In [3]:

# Now we are ready to regress. Start by popping out only the last responded QIDS, and mood
week_freq =  4
epoch1 = 24*3600*(datetime.datetime(2020, 4, 1, 0, 0).timestamp()//(24*3600))*1000
epoch2 = 24*3600*(datetime.datetime(2020, 12, 1, 0, 0).timestamp()//(24*3600))*1000


const = 24*3600*1000*7*week_freq
week_bins = np.arange(epoch1, epoch2 + const, const)
nweeks = len(week_bins)

valid_ids = [key for key in db if  #db[key]['healthcare_worker'] and\
            len(db[key]['QIDS']) > 0 and\
           # db[key]['country'] == 'United States of America' and\
           # db[key]['state'] != 'Puerto Rico' and\
            len(db[key]['A']) > 0 ]

rdb = {key:db[key] for key in valid_ids}
print(len(valid_ids)) # 3

rdict = dict()

rdict['gender'] = []
rdict['age'] = []
rdict['qids'] = []
rdict['mood'] = []
#rdict['camsr'] = []
#rdict['neuroticism'] = []
rdict['pre_physical'] = []
rdict['pre_mental'] = []

for key in valid_ids:
 #   qids_arr = rdb[key]['QIDS']
    pre_mental_values = rdb[key]['pre_mental']
    for val in pre_mental_values:
        pre_mental =  (val=='none of the above')
    #pre_mental = [val in pre_mental if and val is not 'none of the above' and val is not '(empty)']
    pre_physical_values = rdb[key]['pre_physical']
    for val in pre_physical_values:
        pre_physical =  (val=="none of the above" )

    rdict['age'].append(rdb[key]['age'])
    rdict['gender'].append(rdb[key]['gender'])        
    rdict['qids'].append(list(rdb[key]['QIDS'].values())[-1])
    rdict['mood'].append(rdb[key]['A'][-1]['mood'])
 #   rdict['camsr'].append(rdb[key]['camsr'])
 #   rdict['neuroticism'].append(rdb[key]['ipip'][4])
    rdict['pre_physical'].append(1*(pre_physical))
    rdict['pre_mental'].append(1*(pre_mental))

rpd = pd.DataFrame.from_dict(rdict)


740


In [33]:
print('Number of Subjects: %d \n' %len(rpd['qids']))

print('Qids < 10 ')
LowQids = rpd['qids'] < 10
_, p = spearmanr(rpd['qids'][LowQids],rpd['mood'][LowQids])
print('spearman test: (%d Subjects) Coefficient: %.3f P_Value: %.5f \n'% (len(rpd['qids'][LowQids]),_, p))


print('10 <= Qids < 20 ')
HighQids = (rpd['qids'] >= 10) & (rpd['qids'] < 20)
_, p = spearmanr(rpd['qids'][HighQids],rpd['mood'][HighQids])
print('spearman test: (%d Subjects) Coefficient: %.3f P_Value: %.5f \n'% (len(rpd['qids'][HighQids]),_, p))


print('Qids >= 10')
HighQids = (rpd['qids'] >= 10)
_, p = spearmanr(rpd['qids'][HighQids],rpd['mood'][HighQids])
print('spearman test: (%d Subjects) Coefficient: %.3f P_Value: %.5f'% (len(rpd['qids'][HighQids]),_, p))


Number of Subjects: 740 

Qids < 10 
spearman test: (461 Subjects) Coefficient: -0.278 P_Value: 0.00000 

10 <= Qids < 20 
spearman test: (256 Subjects) Coefficient: -0.196 P_Value: 0.00161 

Qids >= 10
spearman test: (279 Subjects) Coefficient: -0.237 P_Value: 0.00007
